In [1]:
import cantera as ct
import cantera.ck2cti
import rmgpy.chemkin
#import numpy as np
import subprocess
import csv
#import scipy
import copy
import os

Function: Load the models

In [2]:
######################################## Load the models ####################################################


#load NIST 2-BTP model, with all the halogens 
#load Nora's 2-BTP model, with all the halogens 

# Load Nora's model (145 species)

full_path_Nora = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/Nora/2_BTP/FFCM_seed/2_BTP_seed/chemkin/'

Nora_chemkin_path = full_path_Nora + 'copies/copy_chem0145.inp'
Nora_dictionary_path = full_path_Nora + 'species_dictionary.txt'
Nora_transport_path = full_path_Nora + 'tran.dat'
Nora_cti_path = full_path_Nora + 'copies/copy_chem0145.cti'

Nora_species_list, Nora_reaction_list = rmgpy.chemkin.load_chemkin_file(Nora_chemkin_path, dictionary_path=Nora_dictionary_path, transport_path=Nora_transport_path)
Nora_gas = ct.Solution(Nora_cti_path)



# # Load David's model 
# full_path_D = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/cantera/RMG_David/'

# D_cti_path = full_path_D + 'chem_annotated.cti'
# D_chemkin_path = full_path_D + 'chem_annotated.inp'
# D_dictionary_path = full_path_D + 'species_dictionary.txt'
# D_transport_path =  full_path_D + 'tran.dat'

# D_gas = ct.Solution(D_cti_path)
# D_dict = rmgpy.chemkin.load_species_dictionary(D_dictionary_path)
# D_species_list, D_reaction_list = rmgpy.chemkin.load_chemkin_file(D_chemkin_path, dictionary_path= D_dictionary_path, transport_path=D_transport_path)



# Load Nora's model with 130 species

_130_cti_path = full_path_Nora + 'copies/copy_chem0130.cti'
_130_chemkin_path = full_path_Nora + 'copies/copy_chem0130.inp'
_130_dictionary_path = full_path_Nora + 'species_dictionary.txt'

_130_gas = ct.Solution(_130_cti_path)
_130_dict = rmgpy.chemkin.load_species_dictionary(_130_dictionary_path)
_130_species_list, _130_reaction_list = rmgpy.chemkin.load_chemkin_file(_130_chemkin_path, dictionary_path=_130_dictionary_path)

Find the reaction to add and the reaction to delete

In [11]:
rxn_to_add = ['C#CC(5272) <=> H(8) + C3H3(6822)']
rxn_to_delete = ['H(8) + C3H3(6822) <=> C#CC(5272)']

for rxn in _130_reaction_list: 
        if str(rxn) == rxn_to_add[0]:
            rxn_to_add.append(rxn.index)
            rxn_to_add.append(rxn)
            print(str(rxn))

for rxn in Nora_reaction_list: 
    if str(rxn) == rxn_to_delete[0]:
        rxn_to_delete.append(rxn.index)
        print(str(rxn))
    
    
print(rxn_to_add)
print(rxn_to_delete)

C#CC(5272) <=> H(8) + C3H3(6822)
H(8) + C3H3(6822) <=> C#CC(5272)
['C#CC(5272) <=> H(8) + C3H3(6822)', 1287, LibraryReaction(index=1287, reactants=[Species(index=5272, label="C#CC", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[3.30522,0.0109266,1.31984e-05,-2.25157e-08,8.87389e-12,20738.2,7.19167], Tmin=(100,'K'), Tmax=(969.99,'K')), NASAPolynomial(coeffs=[5.80059,0.0113535,-4.03487e-06,7.19054e-10,-5.02177e-14,19749.9,-7.36988], Tmin=(969.99,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K')), molecule=[Molecule(smiles="C#CC")], molecular_weight=(40.0638,'amu'))], products=[Species(index=8, label="H", thermo=NASA(polynomials=[NASAPolynomial(coeffs=[2.5,4.63019e-14,-6.5121e-17,3.00122e-20,-4.26132e-24,25474.2,-0.444973], Tmin=(100,'K'), Tmax=(3821.96,'K')), NASAPolynomial(coeffs=[2.5,2.03348e-10,-7.42469e-14,1.19914e-17,-7.22693e-22,25474.2,-0.444972], Tmin=(3821.96,'K'), Tmax=(5000,'K'))], Tmin=(100,'K'), Tmax=(5000,'K')), molecule=[Molecule(smiles="[H]")], molecular_weight=(

Add in desired reaction and delete undesired reaction

In [15]:
desired_reaction = rxn_to_add[-1]

############ delete the undesired rxn. ##############
new_reactions = []

print(len(Nora_reaction_list))
for rxn in Nora_reaction_list:
    if rxn.index == rxn_to_delete[1]:
        print(rxn)
    else: 
        new_reactions.append(rxn) 
print(len(new_reactions)) #this should be one less than Nora_reaction_list
        
new_reactions.append(desired_reaction)

print(len(new_reactions))

1468
H(8) + C3H3(6822) <=> C#CC(5272)
1467
1468


Save the chemkin file

In [ ]:
 #save the chemkin file 

handpicked_dir = '/work/westgroup/nora/Code/projects/halogens/refrigerants/singles/Burgess_Comments/compare/handpicked/add_delete'


chemkin_file = os.path.join(handpicked_dir, 'add_delete_1468.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, Nora_species_list, new_reactions, verbose=True, check_for_duplicates=True)
